In [48]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import pandas as pd
import scipy as sp
import os
import librosa
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
import cv2
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json
from keras.models import load_model
from PIL import Image
import csv

In [49]:
labels = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "silence", "unknown"]

In [47]:
data = []
for label in labels : 
    label_data = np.load(label + ".npy")
    for arr in label_data:
        D = np.abs(librosa.stft(arr))**2
        S = librosa.feature.melspectrogram(S=D, sr=16000, n_mels=128, fmax=8000)
        K = librosa.power_to_db(S,ref=np.max)
        temp = [[label, K]]
        data += temp
data = pd.DataFrame(data, columns=["Label", "Spectrogram"])

KeyboardInterrupt: 

In [8]:
os.mkdir("training_set")
dir_path = "training_set"
for i in labels:
    os.mkdir(os.path.join(dir_path, i))
for i in range(data.shape[0]):
    label = data.iloc[i].Label
    spectrogram = data.iloc[i].Spectrogram
    plt.imsave(os.path.join(dir_path, label, str(i) + ".png"), spectrogram)
    

In [50]:
resimages = [] 
reslabels = []
dir_path = "training_set"
k=0

for label in os.listdir(dir_path):
    for file in os.listdir(os.path.join(dir_path, label)):
        img = cv2.imread(os.path.join(dir_path, label, file))
#         print(img.shape)
        resimages.append(img)
        reslabels.append(label)
        k+=1

np.save("reslabels", reslabels)
np.save("resimages", resimages)

In [69]:
resimages = np.load("resimages.npy")
reslabels = np.load("reslabels.npy")
le = LabelEncoder()
reslabels = le.fit_transform(reslabels)
resimages = np.array(resimages)
reslabels = np.asarray(reslabels)
print(le.inverse_transform(reslabels))
dict = {}
for i in np.unique(reslabels):
    dict[i] = le.inverse_transform([i])
    
print(dict)

['down' 'down' 'down' ... 'yes' 'yes' 'yes']
{0: array(['down'], dtype='<U7'), 1: array(['go'], dtype='<U7'), 2: array(['left'], dtype='<U7'), 3: array(['no'], dtype='<U7'), 4: array(['off'], dtype='<U7'), 5: array(['on'], dtype='<U7'), 6: array(['right'], dtype='<U7'), 7: array(['silence'], dtype='<U7'), 8: array(['stop'], dtype='<U7'), 9: array(['unknown'], dtype='<U7'), 10: array(['up'], dtype='<U7'), 11: array(['yes'], dtype='<U7')}


In [52]:
xtr, xte, ytr, yte = model_selection.train_test_split(resimages, reslabels, test_size=0.2)
def create_model() :
    model = models.Sequential()
    model.add(layers.Conv2D(128 , (3,3), activation='softmax', input_shape=(128, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128 , (3,3), activation='softmax', input_shape=(128, 32, 3)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(12))
    
    return model

In [53]:
print(reslabels.shape)
print(resimages.shape)
model = create_model()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

history = model.fit(resimages, reslabels, epochs=10,  batch_size=100 )

(28176,)
(28176, 128, 32, 3)
Epoch 1/10
282/282 [==============================] - 351s 1s/step - loss: 2.0420 - accuracy: 0.2939
Epoch 2/10
282/282 [==============================] - 328s 1s/step - loss: 1.0819 - accuracy: 0.6513
Epoch 3/10
282/282 [==============================] - 321s 1s/step - loss: 0.8855 - accuracy: 0.7089
Epoch 4/10
282/282 [==============================] - 320s 1s/step - loss: 0.7670 - accuracy: 0.7488
Epoch 5/10
282/282 [==============================] - 305s 1s/step - loss: 0.7010 - accuracy: 0.7706
Epoch 6/10
282/282 [==============================] - 300s 1s/step - loss: 0.6162 - accuracy: 0.7966
Epoch 7/10
282/282 [==============================] - 301s 1s/step - loss: 0.5751 - accuracy: 0.8134
Epoch 8/10
282/282 [==============================] - 300s 1s/step - loss: 0.5413 - accuracy: 0.8218
Epoch 9/10
282/282 [==============================] - 1129s 4s/step - loss: 0.4961 - accuracy: 0.8372
Epoch 10/10
282/282 [==============================] - 338s 1